<https://www.youtube.com/watch?v=9aYuQmMJvjA&list=PLQVvvaa0QuDdeMyHEYc0gxFpYwHY2Qfdh&index=5>

dataset url <https://download.microsoft.com/download/3/E/1/3E1C3F21-ECDB-4869-8368-6DEBA77B919F/kagglecatsanddogs_3367a.zip>

In [ ]:
import os
import cv2
import numpy as np
from tqdm import tqdm


datapath = os.path.abspath(os.path.join(os.getcwd(), '../../__data'))
REBUILD_DATA = False


class DogsVSCats():
    IMAGE_SIZE = 50
    CATS = os.path.join(datapath, "PetImages/Cat")
    DOGS = os.path.join(datapath, "PetImages/Dog")

    LABELS = {
        CATS:0, 
        DOGS: 1,
    }

    training_data = []

    catcount = 0
    dogcount = 0

    def make_training_data(self):
        for label in self.LABELS:
            print(label)
            for f in tqdm(os.listdir(label)):
                try:
                    path = os.path.join(label, f)
                    image = cv2.imread(path, cv2.IMREAD_GRAYSCALE)
                    image = cv2.resize(image, (self.IMAGE_SIZE, self.IMAGE_SIZE))
                    self.training_data.append([np.array(image), np.eye(2)[self.LABELS[label]]])
                    
                    if label == self.CATS:
                        self.catcount += 1
                    elif label == self.DOGS:
                        self.dogcount += 1
                except Exception as e:
                    # print(e)
                    pass
        np.random.shuffle(self.training_data)
        np.save('training_data.npy', self.training_data)
        print('Cat:', self.catcount)
        print('Dog:', self.dogcount)


if REBUILD_DATA:
    dvc = DogsVSCats()
    dvc.make_training_data()

train_data = np.load('training_data.npy', allow_pickle=True)

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class Net(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(1, 32, 5)
        self.conv2 = nn.Conv2d(32, 64, 5)
        self.conv3 = nn.Conv2d(64, 128, 5)

        x = torch.randn(50, 50).view(-1, 1, 50, 50)
        self._to_linear = None
        self.convs(x)

        self.fc1 = nn.Linear(self._to_linear, 512)
        self.fc2 = nn.Linear(512, 2)

    def convs(self, x):
        x = F.max_pool2d(F.relu(self.conv1(x)), 2, 2)
        x = F.max_pool2d(F.relu(self.conv2(x)), 2, 2)
        x = F.max_pool2d(F.relu(self.conv3(x)), 2, 2)
        print(x[0].shape)
        if self._to_linear is None:
            self._to_linear = x[0].shape[0] * x[0].shape[1] * x[0].shape[2]
        return x

    def forward(self, x):
        x = self.convs(x)
        x = x.view(-1, self._to_linear)
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return F.softmax(x, dim=1)


net = Net()

In [ ]:
import torch.optim as optim

optimizer = optim.Adam(net.parameters(), lr=0.001)
loss_function = nn.MSELoss()

x = torch.Tensor([i[0] for i in tqdm(train_data)]).view(-1, 50, 50)
x = x / 255.0
y = torch.Tensor([i[1] for i in tqdm(train_data)])

VAL_PCT = 0.1

VAL_SIZE = int(len(x) * VAL_PCT)

print(VAL_SIZE)

In [ ]:
train_x = x[:-VAL_SIZE]
train_y = y[:-VAL_SIZE]

test_x = x[-VAL_SIZE:]
test_y = y[-VAL_SIZE:]

print(len(train_x))
print(len(test_x))

In [ ]:
BATCH_SIZE = 100

EPOCHS = 1

for epoch in range(EPOCHS):
    for i in tqdm(range(0, len(train_x), BATCH_SIZE)):
        # print(i, i+BATCH_SIZE)
        batch_x = train_x[i: i+ BATCH_SIZE].view(-1, 1, 50, 50)
        batch_y = train_y[i: i + BATCH_SIZE]

        net.zero_grad()

        outputs = net(batch_x)
        loss = loss_function(outputs, batch_y)
        loss.backward()
        optimizer.step()

print(loss)


In [ ]:
correct = 0
total = 0

with torch.no_grad():
    for i in tqdm(range(len(test_x))):
        real_class = torch.argmax(test_y[i])
        net_out = net(test_x[i].view(-1, 1, 50, 50))[0]
        predicted_class = torch.argmax(net_out)
        if predicted_class == real_class:
            correct +=1 
        total += 1

print("Accuracy:", correct / total)